# Delay to get the first SoH
We want to know how much time we have to wait before getting a SoH for a vin


In [ ]:
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
import plotly.express as px
from core.stats_utils import *
from core.sql_utils import *
import numpy as np
import pandas as pd
settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)

s3 = S3Service()


## Load data

In [ ]:
phase_renault = s3.read_parquet_df_spark(spark, "result_phases/result_phases_renault.parquet").toPandas()

In [ ]:
phase_renault.columns


In [ ]:

from core.sql_utils import *
with get_connection() as con:
    cursor = con.cursor()
    cursor.execute("""SELECT v.vin as VIN, m.make_name as MAKE, vm.model_name as MODEL, vd.soh as SOH_prod, vd.timestamp  FROM vehicle_data vd 
                   left JOIN vehicle v on v.id = vd.vehicle_id
                    left JOIN vehicle_model vm
                    ON v.vehicle_model_id = vm.id
                    left JOIN make m on m.id = vm.make_id
                    Where vd.soh < 1.2""")
    dbeaver_df = cursor.fetchall()
dbeaver_df = pd.DataFrame(dbeaver_df, columns=[desc[0] for desc in cursor.description])




## delay evaluation

### Renault

In [ ]:
vin = phase_renault[phase_renault['VIN'] == 'VF1AG000964802627']


In [ ]:
vin = pd.merge_asof(vin.sort_values('DATETIME_BEGIN'),
                    dbeaver_df[['timestamp', 'soh_prod', 'vin']].sort_values('timestamp'), 
                    right_on='timestamp',
                    left_on='DATETIME_BEGIN',
                    left_by='VIN',
                    right_by='vin',
                    direction='forward')

### Filtres temporels

Création de différents filtres pour analyser l'évolution dans le temps depuis la première date de suivi.


In [ ]:
import pandas as pd

# Identifier la première date de suivi
first_date = vin['DATETIME_BEGIN'].min()

# Créer les différents filtres temporels
filters = {
    '1_jour': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(days=1)],
    '1_semaine': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=1)],
    '2_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=2)],
    '3_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=3)],
    '4_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=4)],
    '5_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=5)],
    '6_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=6)],
    '7_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=7)],
    '8_semaines': vin[vin['DATETIME_BEGIN'] <= first_date + pd.Timedelta(weeks=8)],
}



### Option alternative : Filtres par période spécifique (non cumulatifs)

Si vous voulez uniquement les données d'une période spécifique (par exemple seulement la 2ème semaine), utilisez ce code :


In [ ]:
from core.stats_utils import (
    mask_out_outliers_by_interquartile_range,
)

In [ ]:
def make_soh_presentable_per_vehicle( df: pd.DataFrame) -> pd.DataFrame:
    if df["SOH"].count() > 3:
        outliser_mask = mask_out_outliers_by_interquartile_range(df["SOH"])
        assert outliser_mask.any(), (
            f"There seems to be only outliers???:\n{df['SOH']}."
        )
        df = df[outliser_mask].copy()
    return df

In [ ]:
# Filtres non cumulatifs : uniquement les données de chaque période spécifique
filters_specific = {
    '1_jour': vin[
        (vin['DATETIME_BEGIN'] >= first_date) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(days=1))
    ],
    'semaine_1': vin[
        (vin['DATETIME_BEGIN'] >= first_date) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=1))
    ],
    'semaine_2': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=1)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=2))
    ],
    'semaine_3': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=2)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=3))
    ],
    'semaine_4': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=3)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=4))
    ],
    'semaine_5': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=4)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=5))
    ],
    'semaine_6': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=5)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=6))
    ],
    'semaine_7': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=6)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=7))
    ],
    'semaine_8': vin[
        (vin['DATETIME_BEGIN'] >= first_date + pd.Timedelta(weeks=7)) & 
        (vin['DATETIME_BEGIN'] < first_date + pd.Timedelta(weeks=8))
    ],
}

# Afficher le nombre de lignes par période spécifique
print(f"\nNombre de lignes par période spécifique (non cumulatif) :")
for period, df in filters_specific.items():
    print(f"  {period:15s}: {len(df):5d} lignes")


### Visualisation de l'évolution du nombre de données


In [ ]:
# Ordre des périodes
periods_order = [
    '1_jour', '1_semaine', '2_semaines', '3_semaines', '4_semaines',
    '5_semaines', '6_semaines', '7_semaines', '8_semaines'
]

# Création du DataFrame d'évolution
evolution_data = pd.DataFrame({
    'Période': periods_order,
    'SoH median': [make_soh_presentable_per_vehicle(filters[p])['SOH'].median() for p in periods_order],
    'SoH prod': [filters[p].soh_prod.min() for p in periods_order],
    'Jours': [1, 7, 14, 21, 28, 35, 42, 49, 56]
})

# Graphique principal
fig = px.scatter(
    evolution_data,
    x='Jours',
    y='SoH median',
    title='Évolution du SoH médian depuis le début du suivi',
    labels={'Jours': 'Jours depuis le début du suivi', 'SoH median': 'SoH médian'}
)

# Ajout d'une deuxième trace (SoH production médian)
fig.add_scatter(
    x=evolution_data['Jours'],
    y=evolution_data['SoH prod'],
    mode='lines+markers',
    name='SoH prod'
)

# Mise en forme
fig.update_traces(marker=dict(size=10), line=dict(width=3))
fig.update_layout(template='plotly_white', hovermode='x unified')

# Affichage
fig.show()


In [ ]:
# Calcul des statistiques pour chaque tranche temporelle
stats_per_period = []

for period, df in filters_specific.items():
    if len(df) > 0 and 'SOH' in df.columns:
        # Filtrage des outliers
        df_clean = make_soh_presentable_per_vehicle(df) if df["SOH"].count() > 3 else df
        df_val = df_clean.copy()
        df_val['SOH'] = df_val['SOH'] * 100
        df_val['soh_prod'] = df_val['soh_prod'] * 100
        stats = {
            'Période': period,
            'Nombre de données': len(df),
            'Moyenne': df_val['SOH'].mean(),
            'Médiane': df_val['SOH'].median(),
            'Variance': df_val['SOH'].var(),
            'Écart-type': df_val['SOH'].std(),
            'Min': df_val['SOH'].min(),
            'Max': df_val['SOH'].max(),
            'soh_prod': df_val['soh_prod'].min() if 'soh_prod' in df_val.columns else None
        }
    else:
        stats = {
            'Période': period,
            'Nombre de données': 0,
            'Moyenne': None,
            'Médiane': None,
            'Variance': None,
            'Écart-type': None,
            'Min': None,
            'Max': None,
        }
    stats_per_period.append(stats)

# Création du DataFrame des statistiques
df_stats = pd.DataFrame(stats_per_period)

# Tri selon l'ordre des périodes
period_order = ['1_jour', 'semaine_1', 'semaine_2', 'semaine_3', 'semaine_4', 
                'semaine_5', 'semaine_6', 'semaine_7', 'semaine_8']
df_stats['Période'] = pd.Categorical(df_stats['Période'], categories=period_order, ordered=True)
df_stats = df_stats.sort_values('Période')

display(df_stats)


In [ ]:
# Visualisation de l'évolution des statistiques par tranche temporelle
import plotly.graph_objects as go

# Préparation des données pour le graphique
df_stats_clean = df_stats.dropna()  # Retirer les périodes sans données

# Création d'un mapping de période vers jours
period_to_days = {
    '1_jour': 1,
    'semaine_1': 7,
    'semaine_2': 14,
    'semaine_3': 21,
    'semaine_4': 28,
    'semaine_5': 35,
    'semaine_6': 42,
    'semaine_7': 49,
    'semaine_8': 56
}

df_stats_clean['Jours'] = df_stats_clean['Période'].map(period_to_days)

# Création du graphique
fig = go.Figure()

# Zone d'incertitude (min-max)
fig.add_trace(go.Scatter(
    x=df_stats_clean['Jours'],
    y=df_stats_clean['Max'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Max: %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df_stats_clean['Jours'],
    y=df_stats_clean['Min'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.2)',
    fill='tonexty',
    name='Intervalle Min-Max',
    hovertemplate='Min: %{y:.2f}<extra></extra>'
))

# Moyenne
fig.add_trace(go.Scatter(
    x=df_stats_clean['Jours'],
    y=df_stats_clean['Moyenne'],
    mode='lines+markers',
    name='Moyenne',
    line=dict(color='blue', width=3),
    marker=dict(size=10),
    hovertemplate='Moyenne: %{y:.2f}<extra></extra>'
))

# Médiane
fig.add_trace(go.Scatter(
    x=df_stats_clean['Jours'],
    y=df_stats_clean['Médiane'],
    mode='lines+markers',
    name='Médiane',
    line=dict(color='red', width=3, dash='dash'),
    marker=dict(size=10),
    hovertemplate='Médiane: %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df_stats_clean['Jours'],
    y=df_stats_clean['soh_prod'],
    mode='lines+markers',
    name='SoH Prod',
    line=dict(color='green', width=3, dash='dash'),
    marker=dict(size=10),
    hovertemplate='SoH prod: %{y:.2f}<extra></extra>'
))

# Mise en forme
fig.update_layout(
    title='Évolution des statistiques du SoH par tranche temporelle',
    xaxis_title='Jours depuis le début du suivi',
    yaxis_title='SoH (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()


## Analyse de la variance par tranche pour tous les VIN


In [ ]:
all_vehicles = pd.merge_asof(
    phase_renault.sort_values('DATETIME_BEGIN'),
    dbeaver_df[['timestamp', 'soh_prod', 'vin']].sort_values('timestamp'), 
    right_on='timestamp',
    left_on='DATETIME_BEGIN',
    left_by='VIN',
    right_by='vin',
    direction='forward'
)


In [ ]:

def compute_soh_stats_for_vehicle(vin_df):
    """
    Calcule les statistiques du SoH pour un véhicule donné sur différentes périodes
    depuis la première date de suivi.
    
    Args:
        vin_df: DataFrame contenant les données d'un seul véhicule
        
    Returns:
        DataFrame avec les statistiques par période
    """
    if len(vin_df) == 0 or 'SOH' not in vin_df.columns:
        return None
    
    # Identifier la première date de suivi
    first_date = vin_df['DATETIME_BEGIN'].min()
    
    # Définir les périodes à analyser (en jours)
    periods = {
        '1_jour': 1,
        '1_semaine': 7,
        '2_semaines': 14,
        '3_semaines': 21,
        '4_semaines': 28,
        '5_semaines': 35,
        '6_semaines': 42,
        '7_semaines': 49,
        '8_semaines': 56
    }
    
    # Créer les filtres non cumulatifs
    filters_specific = {}
    prev_days = 0
    for period_name, days in periods.items():
        filters_specific[period_name] = vin_df[
            (vin_df['DATETIME_BEGIN'] >= first_date + pd.Timedelta(days=prev_days)) & 
            (vin_df['DATETIME_BEGIN'] < first_date + pd.Timedelta(days=days))
        ]
        prev_days = days
    
    # Calculer les statistiques pour chaque période
    stats_per_period = []
    
    for period, df in filters_specific.items():
        if len(df) > 0:
            # Filtrage des outliers si assez de données
            df_clean = make_soh_presentable_per_vehicle(df) if df["SOH"].count() > 3 else df
            df_clean['SOH'] = df_clean['SOH'] * 100
            if len(df_clean) > 0:
                stats = {
                    'Période': period,
                    'Nombre de données': len(df),
                    'Moyenne': df_clean['SOH'].mean(),
                    'Médiane': df_clean['SOH'].median(),
                    'Variance': df_clean['SOH'].var(),
                    'Écart-type': df_clean['SOH'].std(),
                    'Min': df_clean['SOH'].min(),
                    'Max': df_clean['SOH'].max(),
                    'SoH prod median': df['soh_prod'].median() if 'soh_prod' in df.columns else None
                }
            else:
                stats = {
                    'Période': period,
                    'Nombre de données': len(df),
                    'Moyenne': None,
                    'Médiane': None,
                    'Variance': None,
                    'Écart-type': None,
                    'Min': None,
                    'Max': None,
                    'SoH prod median': None
                }
        else:
            stats = {
                'Période': period,
                'Nombre de données': 0,
                'Moyenne': None,
                'Médiane': None,
                'Variance': None,
                'Écart-type': None,
                'Min': None,
                'Max': None,
                'SoH prod median': None
            }
        stats_per_period.append(stats)
    
    return pd.DataFrame(stats_per_period)

In [ ]:
all_stats = []

vins = all_vehicles['VIN'].unique()
print(f"Traitement de {len(vins)} véhicules...")

for i, vin in enumerate(vins):
    if (i + 1) % 50 == 0:
        print(f"Progression : {i+1}/{len(vins)} véhicules traités")
    
    vin_df = all_vehicles[all_vehicles['VIN'] == vin]
    stats_df = compute_soh_stats_for_vehicle(vin_df)
    
    if stats_df is not None:
        stats_df['VIN'] = vin
        all_stats.append(stats_df)

# Concaténer tous les résultats
all_stats_df = pd.concat(all_stats, ignore_index=True)
all_stats_df_no_na = all_stats_df.dropna()


In [ ]:
all_stats_df_no_na

In [ ]:

period_to_days = {
    '1_jour': 1,
    '1_semaine': 7,
    '2_semaines': 14,
    '3_semaines': 21,
    '4_semaines': 28,
    '5_semaines': 35,
    '6_semaines': 42,
    '7_semaines': 49,
    '8_semaines': 56
}


all_stats_df_no_na['Jours'] = all_stats_df_no_na['Période'].map(period_to_days)


period_order = ['1_jour', '1_semaine', '2_semaines', '3_semaines', '4_semaines', 
                '5_semaines', '6_semaines', '7_semaines', '8_semaines']
all_stats_df_no_na['Période'] = pd.Categorical(all_stats_df_no_na['Période'], 
                                                 categories=period_order, ordered=True)


In [ ]:
# Statistiques agrégées de la variance par période
variance_stats = all_stats_df_no_na.groupby('Période').agg({
    'Variance': ['mean', 'median', 'std', 'min', 'max', 
                 lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    'Écart-type': ['mean', 'median'],
    'Nombre de données': 'sum',
    'VIN': 'nunique'
}).reset_index()

# Aplatir les colonnes
variance_stats.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                          for col in variance_stats.columns.values]
variance_stats = variance_stats.rename(columns={
    'Variance_<lambda_0>': 'Variance_q25',
    'Variance_<lambda_1>': 'Variance_q75'
})

variance_stats['Jours'] = variance_stats['Période'].map(period_to_days)


variance_stats['Période'] = pd.Categorical(variance_stats['Période'], 
                                            categories=period_order, ordered=True)
variance_stats = variance_stats.sort_values('Période')


variance_stats


In [ ]:
# Visualisation de l'évolution de la variance avec intervalles de confiance
fig = go.Figure()

# Zone d'incertitude (Q25 - Q75)
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_q75'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Q75: %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_q25'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    name='Intervalle interquartile',
    hovertemplate='Q25: %{y:.2f}<extra></extra>'
))

# Zone d'incertitude (Moyenne ± 1 écart-type)
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_mean'] + variance_stats['Variance_std'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hovertemplate='Moyenne + 1σ: %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_mean'] - variance_stats['Variance_std'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(0, 0, 255, 0.15)',
    fill='tonexty',
    name='Moyenne ± 1 écart-type',
    hovertemplate='Moyenne - 1σ: %{y:.2f}<extra></extra>'
))

# Moyenne de la variance
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_mean'],
    mode='lines+markers',
    name='Moyenne de la variance',
    line=dict(color='blue', width=4),
    marker=dict(size=12),
    hovertemplate='Moyenne<br>Jours: %{x}<br>Variance: %{y:.2f}<extra></extra>'
))

# Médiane de la variance
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_median'],
    mode='lines+markers',
    name='Médiane de la variance',
    line=dict(color='red', width=4, dash='dash'),
    marker=dict(size=12),
    hovertemplate='Médiane<br>Jours: %{x}<br>Variance: %{y:.2f}<extra></extra>'
))

# Min et Max
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_max'],
    mode='markers',
    name='Maximum',
    marker=dict(symbol='triangle-up', size=8, color='orange'),
    hovertemplate='Max: %{y:.2f}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_min'],
    mode='markers',
    name='Minimum',
    marker=dict(symbol='triangle-down', size=8, color='green'),
    hovertemplate='Min: %{y:.2f}<extra></extra>'
))

fig.update_layout(
    title='Évolution de la variance du SoH par période (tous les VINs)',
    xaxis_title='Jours depuis le début du suivi',
    yaxis_title='Variance du SoH',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=600
)

fig.show()


## Identification de la période la plus performante


In [ ]:
# Identifier la période avec la variance moyenne la plus faible (plus performante)

best_period_by_median_variance = variance_stats.loc[variance_stats['Variance_median'].idxmin()]



In [ ]:

fig = go.Figure()


colors_bar = ['green' if p == best_period_by_median_variance['Période'] else 'lightblue' 
              for p in variance_stats['Période']]

fig.add_trace(go.Bar(
    x=variance_stats['Période'],
    y=variance_stats['Variance_median'],
    name='Variance moyenne',
    marker=dict(color=colors_bar),
    hovertemplate='Période: %{x}<br>Variance médiane: %{y:.2f}<extra></extra>'
))

# Ligne de référence pour la variance moyenne minimale
fig.add_hline(
    y=variance_stats['Variance_median'].mean(),
    line_dash="dash",
    line_color="green",
    annotation_text=f"Mean: {variance_stats['Variance_median'].mean():.2f}",
    annotation_position="right"
)

fig.update_layout(
    title=f'Variance par période ',
    xaxis_title='Période',
    yaxis_title='Variance',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()


In [ ]:
# Analyse de l'écart-type (plus interprétable que la variance)
ecart_type_stats = all_stats_df_no_na.groupby('Période').agg({
    'Écart-type': ['mean', 'median', 'std', 'min', 'max'],
    'Variance': ['mean', 'median'],
    'VIN': 'nunique'
}).reset_index()

ecart_type_stats.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                            for col in ecart_type_stats.columns.values]
ecart_type_stats['Jours'] = ecart_type_stats['Période'].map(period_to_days)
ecart_type_stats['Période'] = pd.Categorical(ecart_type_stats['Période'], 
                                               categories=period_order, ordered=True)
ecart_type_stats = ecart_type_stats.sort_values('Période')

# Identifier la période avec l'écart-type moyen le plus faible
best_period_by_std = ecart_type_stats.loc[ecart_type_stats['Variance_mean'].idxmin()]



In [ ]:
# Visualisation de l'évolution de l'écart-type avec mise en évidence de la meilleure période
fig = go.Figure()

colors_bar = ['green' if p == best_period_by_std['Période'] else 'lightblue' 
              for p in ecart_type_stats['Période']]

fig.add_trace(go.Bar(
    x=ecart_type_stats['Période'],
    y=ecart_type_stats['Écart-type_mean'],
    name='Écart-type mean',
    marker=dict(color=colors_bar),
    hovertemplate='Période: %{x}<br>Écart-type mean: %{y:.2f}%<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=ecart_type_stats['Période'],
    y=ecart_type_stats['Écart-type_median'],
    mode='lines+markers',
    name='Écart-type médian',
    line=dict(color='orange', width=3),
    marker=dict(size=10),
    hovertemplate='Période: %{x}<br>Écart-type médian: %{y:.2f}%<extra></extra>'
))

fig.add_hline(
    y=ecart_type_stats['Écart-type_median'].min(),
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean: {ecart_type_stats['Écart-type_median'].mean():.2f}%",
    annotation_position="right"
)

fig.update_layout(
    title=f'Écart-type par période',
    xaxis_title='Période',
    yaxis_title='Écart-type (%)',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()


In [ ]:
# Résumé final de la performance par période


# Créer un tableau récapitulatif
summary_performance = variance_stats[['Période', 'Jours', 'Variance_mean', 'Variance_median', 
                                     'Écart-type_mean', 'VIN_nunique']].copy()
summary_performance['Rang_variance_moyenne'] = summary_performance['Variance_mean'].rank()
summary_performance['Rang_ecart_type'] = summary_performance['Écart-type_mean'].rank()
summary_performance = summary_performance.sort_values('Rang_variance_moyenne')

display(summary_performance.round(2))


## Analyse des causes de la variance élevée observée sur certains véhicules spécifiques

### 1. Identification des véhicules avec variance élevée


In [ ]:
# Identifier les véhicules avec variance élevée par période
# On se concentre sur les périodes de 4 semaines et plus où on devrait avoir une bonne précision
periods_to_analyze = ['4_semaines', '5_semaines', '6_semaines', '7_semaines', '8_semaines']

# Filtrer les données pour ces périodes
high_variance_df = all_stats_df_no_na[
    all_stats_df_no_na['Période'].isin(periods_to_analyze)
].copy()

# Calculer la variance moyenne par véhicule sur ces périodes
vin_variance_stats = high_variance_df.groupby('VIN').agg({
    'Variance': ['mean', 'max', 'std'],
    'Écart-type': ['mean', 'max'],
    'Nombre de données': 'sum',
    'Moyenne': 'mean',
    'Médiane': 'mean',
    'Min': 'min',
    'Max': 'max'
}).reset_index()

# Aplatir les colonnes
vin_variance_stats.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                              for col in vin_variance_stats.columns.values]

# Définir un seuil pour variance élevée (75e percentile)
variance_threshold = vin_variance_stats['Variance_mean'].quantile(0.75)
std_threshold = vin_variance_stats['Écart-type_mean'].quantile(0.75)

print(f"Seuil de variance moyenne (75e percentile): {variance_threshold:.2f}")
print(f"Seuil d'écart-type moyen (75e percentile): {std_threshold:.2f}")
print(f"\nNombre de véhicules avec variance élevée: {(vin_variance_stats['Variance_mean'] > variance_threshold).sum()}")
print(f"Nombre de véhicules avec écart-type élevé: {(vin_variance_stats['Écart-type_mean'] > std_threshold).sum()}")

# Identifier les véhicules avec variance élevée
high_variance_vins = vin_variance_stats[
    (vin_variance_stats['Variance_mean'] > variance_threshold) |
    (vin_variance_stats['Écart-type_mean'] > std_threshold)
].sort_values('Variance_mean', ascending=False)

print(f"\nTop 10 véhicules avec la variance la plus élevée:")
display(high_variance_vins.head(10)[['VIN', 'Variance_mean', 'Écart-type_mean', 'Nombre de données_sum', 'Moyenne_mean', 'Médiane_mean']])


### 2. Analyse des caractéristiques des véhicules avec variance élevée


In [ ]:
# Comparer les caractéristiques des véhicules avec variance élevée vs normale
normal_variance_vins = vin_variance_stats[
    (vin_variance_stats['Variance_mean'] <= variance_threshold) &
    (vin_variance_stats['Écart-type_mean'] <= std_threshold)
]

comparison_stats = pd.DataFrame({
    'Métrique': [
        'Nombre de véhicules',
        'Variance moyenne',
        'Écart-type moyen',
        'Nombre total de données',
        'Nombre moyen de données par période',
        'Amplitude SoH (Max - Min)',
        'Écart Moyenne-Médiane'
    ],
    'Variance élevée': [
        len(high_variance_vins),
        high_variance_vins['Variance_mean'].mean(),
        high_variance_vins['Écart-type_mean'].mean(),
        high_variance_vins['Nombre de données_sum'].mean(),
        high_variance_vins['Nombre de données_sum'].mean() / len(periods_to_analyze),
        (high_variance_vins['Max_max'] - high_variance_vins['Min_min']).mean(),
        (high_variance_vins['Moyenne_mean'] - high_variance_vins['Médiane_mean']).abs().mean()
    ],
    'Variance normale': [
        len(normal_variance_vins),
        normal_variance_vins['Variance_mean'].mean(),
        normal_variance_vins['Écart-type_mean'].mean(),
        normal_variance_vins['Nombre de données_sum'].mean(),
        normal_variance_vins['Nombre de données_sum'].mean() / len(periods_to_analyze),
        (normal_variance_vins['Max_max'] - normal_variance_vins['Min_min']).mean(),
        (normal_variance_vins['Moyenne_mean'] - normal_variance_vins['Médiane_mean']).abs().mean()
    ]
})

comparison_stats['Ratio'] = comparison_stats['Variance élevée'] / comparison_stats['Variance normale']
display(comparison_stats.round(2))


In [ ]:
# Visualisation comparative : Variance vs Nombre de données
fig = go.Figure()

# Véhicules avec variance normale
fig.add_trace(go.Scatter(
    x=normal_variance_vins['Nombre de données_sum'],
    y=normal_variance_vins['Variance_mean'],
    mode='markers',
    name='Variance normale',
    marker=dict(color='blue', size=8, opacity=0.6),
    hovertemplate='VIN: %{text}<br>Données: %{x}<br>Variance: %{y:.2f}<extra></extra>',
    text=normal_variance_vins['VIN']
))

# Véhicules avec variance élevée
fig.add_trace(go.Scatter(
    x=high_variance_vins['Nombre de données_sum'],
    y=high_variance_vins['Variance_mean'],
    mode='markers',
    name='Variance élevée',
    marker=dict(color='red', size=10, opacity=0.8),
    hovertemplate='VIN: %{text}<br>Données: %{x}<br>Variance: %{y:.2f}<extra></extra>',
    text=high_variance_vins['VIN']
))

# Ligne de seuil
fig.add_hline(
    y=variance_threshold,
    line_dash="dash",
    line_color="orange",
    annotation_text=f"Seuil (75e percentile): {variance_threshold:.2f}",
    annotation_position="right"
)

fig.update_layout(
    title='Relation entre nombre de données et variance du SoH',
    xaxis_title='Nombre total de données (4-8 semaines)',
    yaxis_title='Variance moyenne du SoH',
    template='plotly_white',
    hovermode='closest',
    height=600
)

fig.show()


### 3. Analyse temporelle détaillée pour les véhicules problématiques


In [ ]:
# Analyser l'évolution de la variance dans le temps pour les véhicules problématiques
top_problematic_vins = high_variance_vins.head(5)['VIN'].tolist()

print(f"Analyse détaillée des {len(top_problematic_vins)} véhicules les plus problématiques:\n")

for vin in top_problematic_vins:
    vin_stats = all_stats_df_no_na[all_stats_df_no_na['VIN'] == vin].copy()
    vin_stats = vin_stats.sort_values('Période')
    
    print(f"\n{'='*80}")
    print(f"VIN: {vin}")
    print(f"{'='*80}")
    print(f"Variance moyenne: {vin_stats['Variance'].mean():.2f}")
    print(f"Écart-type moyen: {vin_stats['Écart-type'].mean():.2f}")
    print(f"Nombre total de données: {vin_stats['Nombre de données'].sum()}")
    print(f"\nÉvolution par période:")
    display(vin_stats[['Période', 'Nombre de données', 'Variance', 'Écart-type', 'Moyenne', 'Médiane', 'Min', 'Max']])


In [ ]:
# Visualisation de l'évolution temporelle pour les véhicules problématiques
fig = go.Figure()

for vin in top_problematic_vins:
    vin_stats = all_stats_df_no_na[all_stats_df_no_na['VIN'] == vin].copy()
    vin_stats = vin_stats.sort_values('Jours')
    
    fig.add_trace(go.Scatter(
        x=vin_stats['Jours'],
        y=vin_stats['Variance'],
        mode='lines+markers',
        name=f'VIN: {vin[:8]}...',
        hovertemplate=f'VIN: {vin}<br>Jours: %{{x}}<br>Variance: %{{y:.2f}}<extra></extra>'
    ))

# Ligne de référence (variance moyenne globale)
fig.add_trace(go.Scatter(
    x=variance_stats['Jours'],
    y=variance_stats['Variance_mean'],
    mode='lines',
    name='Variance moyenne (tous véhicules)',
    line=dict(color='black', width=3, dash='dash'),
    hovertemplate='Moyenne globale<br>Jours: %{x}<br>Variance: %{y:.2f}<extra></extra>'
))

fig.update_layout(
    title='Évolution de la variance dans le temps - Véhicules problématiques',
    xaxis_title='Jours depuis le début du suivi',
    yaxis_title='Variance du SoH',
    template='plotly_white',
    hovermode='x unified',
    height=600,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


### 4. Analyse des données brutes pour comprendre les causes


In [ ]:
# Analyser les données brutes pour quelques véhicules problématiques
def analyze_vehicle_raw_data(vin_code, all_vehicles_df):
    """Analyse les données brutes d'un véhicule pour identifier les causes de variance élevée"""
    vin_data = all_vehicles_df[all_vehicles_df['VIN'] == vin_code].copy()
    vin_data = vin_data.sort_values('DATETIME_BEGIN')
    
    # Convertir SoH en pourcentage
    vin_data['SOH_pct'] = vin_data['SOH'] * 100
    
    # Calculer les statistiques
    analysis = {
        'VIN': vin_code,
        'Nombre total de phases': len(vin_data),
        'Période de suivi (jours)': (vin_data['DATETIME_BEGIN'].max() - vin_data['DATETIME_BEGIN'].min()).days,
        'Variance SoH': vin_data['SOH_pct'].var(),
        'Écart-type SoH': vin_data['SOH_pct'].std(),
        'Moyenne SoH': vin_data['SOH_pct'].mean(),
        'Médiane SoH': vin_data['SOH_pct'].median(),
        'Min SoH': vin_data['SOH_pct'].min(),
        'Max SoH': vin_data['SOH_pct'].max(),
        'Amplitude SoH': vin_data['SOH_pct'].max() - vin_data['SOH_pct'].min(),
        'Écart Moyenne-Médiane': abs(vin_data['SOH_pct'].mean() - vin_data['SOH_pct'].median()),
        'Nombre de valeurs uniques SoH': vin_data['SOH_pct'].nunique(),
        'Pourcentage de données manquantes SoH': (vin_data['SOH'].isna().sum() / len(vin_data)) * 100
    }
    
    # Analyser la distribution
    q25, q75 = vin_data['SOH_pct'].quantile([0.25, 0.75])
    analysis['IQR'] = q75 - q25
    
    # Identifier les outliers (méthode IQR)
    outliers = vin_data[(vin_data['SOH_pct'] < q25 - 1.5 * analysis['IQR']) | 
                        (vin_data['SOH_pct'] > q75 + 1.5 * analysis['IQR'])]
    analysis['Nombre outliers (IQR)'] = len(outliers)
    analysis['Pourcentage outliers'] = (len(outliers) / len(vin_data)) * 100
    
    # Analyser les patterns temporels
    vin_data['jours_since_start'] = (vin_data['DATETIME_BEGIN'] - vin_data['DATETIME_BEGIN'].min()).dt.days
    
    # Variance dans les premières semaines vs dernières semaines
    first_4_weeks = vin_data[vin_data['jours_since_start'] < 28]
    after_4_weeks = vin_data[vin_data['jours_since_start'] >= 28]
    
    if len(first_4_weeks) > 0:
        analysis['Variance semaines 1-4'] = first_4_weeks['SOH_pct'].var()
        analysis['Écart-type semaines 1-4'] = first_4_weeks['SOH_pct'].std()
    
    if len(after_4_weeks) > 0:
        analysis['Variance après 4 semaines'] = after_4_weeks['SOH_pct'].var()
        analysis['Écart-type après 4 semaines'] = after_4_weeks['SOH_pct'].std()
    
    return analysis, vin_data

# Analyser les véhicules problématiques
problematic_analyses = []
for vin in top_problematic_vins:
    analysis, _ = analyze_vehicle_raw_data(vin, all_vehicles)
    problematic_analyses.append(analysis)

analysis_df = pd.DataFrame(problematic_analyses)
display(analysis_df.round(2))


In [ ]:
# Visualisation des données brutes pour un véhicule problématique représentatif
vin_to_plot = top_problematic_vins[0]
_, vin_raw_data = analyze_vehicle_raw_data(vin_to_plot, all_vehicles)

# Créer un graphique avec plusieurs sous-graphiques
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'Évolution du SoH dans le temps (VIN: {vin_to_plot[:8]}...)',
        'Distribution du SoH',
        'Variance mobile (fenêtre 7 jours)',
        'Évolution de l\'écart-type mobile'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Graphique 1: Évolution temporelle
vin_raw_data['jours_since_start'] = (vin_raw_data['DATETIME_BEGIN'] - vin_raw_data['DATETIME_BEGIN'].min()).dt.days
fig.add_trace(
    go.Scatter(
        x=vin_raw_data[["SOH_pct", "jours_since_start"]].dropna()['jours_since_start'],
        y=vin_raw_data[["SOH_pct", "jours_since_start"]].dropna()['SOH_pct'],
        mode='markers',
        name='SoH',
        marker=dict(size=4, opacity=0.6),
        hovertemplate='Jour: %{x}<br>SoH: %{y:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# Ajouter une ligne de tendance
from scipy import stats
z = np.polyfit(vin_raw_data[["SOH_pct", "jours_since_start"]].dropna()['jours_since_start'].dropna(), vin_raw_data[["SOH_pct", "jours_since_start"]].dropna()['SOH_pct'].dropna(), 1)
p = np.poly1d(z)
fig.add_trace(
    go.Scatter(
        x=vin_raw_data['jours_since_start'],
        y=p(vin_raw_data['jours_since_start']),
        mode='lines',
        name='Tendance',
        line=dict(color='red', width=2, dash='dash')
    ),
    row=1, col=1
)

# Graphique 2: Distribution
fig.add_trace(
    go.Histogram(
        x=vin_raw_data['SOH_pct'],
        nbinsx=30,
        name='Distribution SoH',
        marker_color='lightblue'
    ),
    row=1, col=2
)

# Ajouter lignes de moyenne et médiane
mean_soh = vin_raw_data['SOH_pct'].mean()
median_soh = vin_raw_data['SOH_pct'].median()
fig.add_vline(x=mean_soh, line_dash="dash", line_color="blue", 
              annotation_text=f"Moyenne: {mean_soh:.2f}", row=1, col=2)
fig.add_vline(x=median_soh, line_dash="dash", line_color="red", 
              annotation_text=f"Médiane: {median_soh:.2f}", row=1, col=2)

# Graphique 3: Variance mobile
window_size = 7
vin_raw_data_sorted = vin_raw_data.sort_values('jours_since_start')
rolling_var = vin_raw_data_sorted['SOH_pct'].rolling(window=window_size, min_periods=1).var()
fig.add_trace(
    go.Scatter(
        x=vin_raw_data_sorted['jours_since_start'],
        y=rolling_var,
        mode='lines',
        name='Variance mobile',
        line=dict(color='orange', width=2)
    ),
    row=2, col=1
)

# Graphique 4: Écart-type mobile
rolling_std = vin_raw_data_sorted['SOH_pct'].rolling(window=window_size, min_periods=1).std()
fig.add_trace(
    go.Scatter(
        x=vin_raw_data_sorted['jours_since_start'],
        y=rolling_std,
        mode='lines',
        name='Écart-type mobile',
        line=dict(color='green', width=2)
    ),
    row=2, col=2
)

# Mise à jour des axes
fig.update_xaxes(title_text="Jours depuis le début", row=1, col=1)
fig.update_yaxes(title_text="SoH (%)", row=1, col=1)
fig.update_xaxes(title_text="SoH (%)", row=1, col=2)
fig.update_yaxes(title_text="Fréquence", row=1, col=2)
fig.update_xaxes(title_text="Jours depuis le début", row=2, col=1)
fig.update_yaxes(title_text="Variance", row=2, col=1)
fig.update_xaxes(title_text="Jours depuis le début", row=2, col=2)
fig.update_yaxes(title_text="Écart-type", row=2, col=2)

fig.update_layout(
    title=f'Analyse détaillée - Véhicule avec variance élevée (VIN: {vin_to_plot})',
    template='plotly_white',
    height=900,
    showlegend=True
)

fig.show()


### 6. Synthèse et hypothèses sur les causes de variance élevée


### Conclusion

- Une grande variabilité entre les différents véhicules, avec des patterns individuels significatifs

**2. Période la plus performante**

L'identification de la période optimale pour les estimations de SoH dépend de la métrique considérée :

- **Variance médiane** : La période de **4 semaines** présente la variance médiane la plus faible, suggérant qu'une majorité de véhicules atteignent une bonne précision plus rapidement
- **Variance moyenne** : La période de **6 semaines** présente la variance médiane la plus faible, suggérant qu'une majorité de véhicules atteignent une bonne précision plus rapidement

L'analyse révèle que :
- Avant 3 semaines la variance est très élevée, notamment due à un manque de données et à la phase d'initialisation du suivi
- La période optimale semble être aux alentours de 4 semaines, où la variance est minimisée
- L'écart-type moyen montre une tendance similaire, confirmant que la précision s'améliore avec le temps

#### Recommandations

**Pour la production :**

1. **Délai minimum recommandé** : Attendre au minimum **3 semaines** de données avant d'utiliser les estimations de SoH afin d'obtenir une précision acceptable pour la majorité des véhicules

2. **Suivi individuel** : L'analyse par VIN montre une grande hétérogénéité. Il est recommandé de surveiller individuellement les véhicules présentant une variance anormalement élevée, même après plusieurs semaines de suivim

**NS :**

- Faire la même hose pour d'autres marques
- Investiguer les causes de la variance élevée observée sur certains véhicules spécifiques

